In [1]:
# Calculate sBERT Embeddings

In [2]:
import sys

sys.version

'3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) \n[GCC 10.3.0]'

In [2]:
def transform_to_sbert_embeddings(input):
    import os
    import json
    import sys
    # I created the venv-environment below using 
    
    #VENV_LIB_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env/lib/python3.8/site-packages'
    #VENV_LIB_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env-python3.7/lib/python3.7/site-packages'
    VENV_LIB_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env-python3.8/lib/python3.8/site-packages'
    if VENV_LIB_DIR not in sys.path:
        sys.path = [VENV_LIB_DIR] + sys.path
    os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/sbert-env/sentence-transformer-home'
    from sentence_transformers import SentenceTransformer
    sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    return json.dumps(sbert_model.encode(input).tolist())

In [3]:
transform_to_sbert_embeddings('hello world foo')

ImportError: Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.

In [3]:
import pyspark

sc = pyspark.SparkContext()
sc

22/02/08 14:21:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/08 14:21:58 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
22/02/08 14:21:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
22/02/08 14:22:14 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


<SparkContext master=yarn appName=pyspark-shell>

In [4]:
def load_queries(file_name):
    ret = []
    import json
    from tqdm import tqdm
    with open(file_name, 'r') as f:
        for l in tqdm(f):
            ret += [json.loads(l)]
    return ret
    
queries = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/msmarco-document-train.jsonl')

367013it [00:05, 73188.42it/s]


In [5]:
queries[:3]

[{'id': '1185869',
  'contents': ')what was the immediate impact of the success of the manhattan project?',
  'target_document': ['D59219']},
 {'id': '1185868',
  'contents': '_________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:',
  'target_document': ['D59235']},
 {'id': '1183785',
  'contents': 'elegxo meaning',
  'target_document': ['D576811']}]

In [5]:
def append_sbert_embedding(i):
    import json
    i['sbert-embedding'] = transform_to_sbert_embeddings(i['contents'])
    return json.dumps(i)

sc.parallelize(queries, 10000)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/embeddings-ms-marco-train.jsonl')

In [7]:
robust04_desc_queries = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/robust04-descriptions.jsonl')

sc.parallelize(robust04_desc_queries, 100)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/embeddings-robust04-descriptions.jsonl')

250it [00:00, 38687.13it/s]


In [8]:
robust04_formulation_queries = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/culpepper-robust04-query-formulations.jsonl')

sc.parallelize(robust04_formulation_queries, 100)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/culpepper-robust04-query-formulations.jsonl')

3151it [00:00, 108014.74it/s]


In [13]:
robust04_queries = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/trec-robust04.jsonl')

sc.parallelize(robust04_queries, 10)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/embeddings-trec-robust04.jsonl')

250it [00:00, 41929.62it/s]


In [15]:
sc.textFile('file:///mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/msmarco-document-orcas.jsonl')\
    .repartition(100)\
    .saveAsTextFile('ecir22/original-ms-marco-orcas.jsonl')

In [6]:
trec18_reformulations = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/culpepper-trec18-query-formulations.jsonl')

sc.parallelize(trec18_reformulations, 10)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/embeddings-trec18-reformulations.jsonl')

1505it [00:00, 71136.36it/s]


In [9]:
trec18_reformulations = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/wapo-core-2018-descriptions.jsonl')

sc.parallelize(trec18_reformulations, 10)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/embeddings-trec18-descriptions.jsonl')

50it [00:00, 10988.48it/s]


In [10]:
trec18_reformulations = load_queries('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-zero-shot/wapo-core-2018-titles.jsonl')

sc.parallelize(trec18_reformulations, 10)\
    .map(append_sbert_embedding)\
    .saveAsTextFile('ecir22/embeddings-trec18-titles.jsonl')

50it [00:00, 12575.12it/s]


In [ ]:
import json

for part in range(10):
    input_file = 'ecir22/original-ms-marco-orcas.jsonl/part-*' + str(part)
    output_file = 'ecir22/embeddings-ms-marco-orcas/part-' + str(part) + '.jsonl'
    
    print('Process "' + input_file + '" and write to "' + output_file +'".')
    sc.textFile(input_file)\
        .repartition(10000) \
        .map(lambda i: append_sbert_embedding(json.loads(i)))\
        .saveAsTextFile(output_file)


Process "ecir22/original-ms-marco-orcas.jsonl/part-*0" and write to "ecir22/embeddings-ms-marco-orcas/part-0.jsonl".
Process "ecir22/original-ms-marco-orcas.jsonl/part-*1" and write to "ecir22/embeddings-ms-marco-orcas/part-1.jsonl".
Process "ecir22/original-ms-marco-orcas.jsonl/part-*2" and write to "ecir22/embeddings-ms-marco-orcas/part-2.jsonl".


In [ ]:
# Run ORCAS simply via writing to hdfs (with 100 parts) and combining all parts again

In [6]:
transform_to_sbert_embeddings('Hello World')

'[-0.08469943702220917, 0.4564870297908783, 0.10086822509765625, -0.11005697399377823, 0.04533920809626579, -0.3661494851112366, 0.3580332398414612, -0.07830145955085754, -0.24362005293369293, 0.2736354470252991, 0.10803011804819107, -0.7733525633811951, -0.13244451582431793, 0.19357416033744812, 0.20357149839401245, -0.36997082829475403, 0.3396165668964386, -0.6763170957565308, -0.849643886089325, -0.19371700286865234, -0.230658158659935, -0.02822048030793667, 0.34298205375671387, 0.20185132324695587, 0.24069765210151672, -0.27264273166656494, 0.07051801681518555, 0.28514087200164795, 0.27905553579330444, -0.010386943817138672, -0.011181384325027466, 0.04527701437473297, 0.24535013735294342, -0.020689629018306732, 0.047592952847480774, 0.48722851276397705, 0.1144338995218277, -0.12156331539154053, -0.15390871465206146, -0.11076535284519196, 0.29678648710250854, -0.4554610550403595, 0.11478982865810394, 0.06491449475288391, -0.2805788516998291, -0.005584864877164364, -0.032436385750770